In [83]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [4]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [5]:
def stopQualityMask(speciesCode: str,closestStop: str):
    cnx = connectDB()
    try:
        gap = dt.date.today().year-2018
        query = f'SELECT COUNT(DISTINCT(year)) as "frq" FROM coefficients_bySpecies WHERE speciesCode = "{speciesCode}" AND closestStop = "{closestStop}"'
        coeficients = pd.read_sql(query,con=cnx)
        coeficients['frq'] = coeficients.apply(lambda g: (g.frq/gap),axis=1)
    except Exception as maskExc:
        raise maskExc
    return coeficients['frq']

In [224]:
def log_stopMetrics(speciesCode,key,testBlob,latestUpdate):
    cnx = connectDB()
    cur = cnx.cursor()
    try:
        sqliteInsert = """INSERT INTO stopMetricsBlob (speciesCode,key,testBlob,latestUpdate) VALUES (?,?,?,?);"""
        logTuple = (speciesCode,key,testBlob,latestUpdate)
        cur.execute(sqliteInsert,logTuple)
        cnx.commit()
        cur.close()
    except db.Error as sqlError:
        raise sqlError
    finally:
        if cnx:
            cnx.close()

In [274]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings


In [275]:
birdlist = birdList_request(StopName='EastSunsetWay',cnx=connectDB())

In [278]:
birdlist[birdlist['speciesCode']=='gockin']

,speciesCode,checklists,sightings,stopGroup,overall,StopName
38,gockin,36,179,4,4,EastSunsetWay


For birds with robust data, do they behave the same at all stops? If not, what stops does each species appear to prefer? Is that preference explained by habitat?

In [264]:
def wklyAbd_selectSpecies(cnx,speciesList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.locId in ({query_locIds})'
        ##todo #83 write an exception to convert non-string speciesList values to strings that returns the db.error if the strings do not represent columns --DONE
        query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','StopName','obsDt_week'])['howMany'].mean().reset_index()
        #maxCount = obsData.groupby(['speciesCode'])['howMany'].max()
        avgCount = obsData.groupby(['speciesCode'])['howMany'].mean()
        #derive relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/(avgCount[x.speciesCode])),axis=1)  #baseline relative abd, same process as Fink et. all
        obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        avgAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
        obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return obsData

In [330]:
##TODO #90 rewrite stopCovariance to run for a single species at a time

def stopCovariance(obsData: pd.DataFrame,species: str,StopKey: str,cnx):
    try:
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
        container = []
        resultsContainer = []
        stopKeys = obsData.drop_duplicates(subset=['StopName'])
        for StopName in stopKeys.itertuples():
            stop_obsData = obsData[obsData['speciesCode']==species]
            stop_obsData = stop_obsData[stop_obsData['StopName']==StopName.StopName]
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            stop_obsData.drop(columns=['speciesCode','StopName','howMany'],inplace=True)
            stop_obsData = pd.merge(left=stop_obsData,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            stop_obsData.set_index('obsDt_week',inplace=True)
            stop_obsData.sort_index(axis='index',ascending=True,inplace=True)
            stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=5,limit_direction='forward')  #mask, values do not matter
            stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')  #mask, values do not matter
            stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
            stop_obsData['fx_relativeAbundance'] = stop_obsData['relativeAbundance'].interpolate(method='linear',limit=5,limit_direction='forward')
            stop_obsData['fx_relativeAbundance'] = stop_obsData['fx_relativeAbundance'].interpolate(method='linear',limit=2,limit_direction='backward')
            stop_obsData.drop(columns=['relativeAbundance','mask'],inplace=True)
            stop_obsData.rename(columns={'fx_relativeAbundance':StopName.StopName},inplace=True)              
            container.append(stop_obsData)
        weeklySpeciesAbd = pd.concat(container,ignore_index=False,axis=1)
        weeklySpeciesAbd = weeklySpeciesAbd.fillna(value=0,axis=0)
        weeklySpeciesAbd = weeklySpeciesAbd.apply(lambda n:(np.log1p(n)),axis=1)
    #return related stops
        fit = [(stop,r2_score(weeklySpeciesAbd.EastSunsetWay,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        rSqr = filter(lambda r: (r[1] > 0.25) and (r[1]<1),fit)
        var = [(stop,explained_variance_score(weeklySpeciesAbd.EastSunsetWay,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        expVar = filter(lambda e: (e[1]>0.25) and (e[1]<1),var)
        #blob = {'rSquared':list(rSqr),'explVar':list(expVar)}
        #log_stopMetrics(speciesCode=species,key=StopKey,testBlob=str(blob),latestUpdate=str(dt.datetime.today()))

    except Exception as ex:
        raise ex
    return {'speciesCode':species,'stopKey':StopKey,'rSquared':list(rSqr),'explVar':list(expVar)}

In [289]:
amerob = wklyAbd_selectSpecies(cnx=connectDB(),speciesList=['gockin'])


In [331]:
tr = stopCovariance(obsData=amerob,species='gockin',StopKey='EastSunsetWay',cnx=connectDB())
tr

{'speciesCode': 'gockin',
 'stopKey': 'EastSunsetWay',
 'rSquared': [('MountTeneriffe', 0.34731861691982624),
  ('SquakMountain', 0.47212864403152854)],
 'explVar': [('MountTeneriffe', 0.3822137768529692),
  ('SquakMountain', 0.47882020057342145)]}